In [3]:
from utils import *

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import folium

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error

#### Carregando o dataset

In [4]:
df_train = pd.read_csv('../data/treated_data/train.csv')
df_val = pd.read_csv('../data/treated_data/validation.csv')
df_test = pd.read_csv('../data/treated_data/test.csv')

Concatenando os datasets de treino e validação, pois o Random Forest não necessita de validação

In [24]:
df_train = pd.concat([df_train, df_val], ignore_index=False)

In [25]:
df_train.head()

,ponto_id,rssi_1_1,rssi_1_2,rssi_1_3,rssi_2_1,rssi_2_2,rssi_2_3,rssi_3_1,rssi_3_2,rssi_3_3,delay_1,delay_2,delay_3,lat,lon,indoor
0,8790,0.559200,0.398652,0.703014,0.267376,0.426916,0.454187,0.317280,0.311331,0.301273,0.333333,0.571429,0.428571,-8.050090,-34.953120,0
1,3227,0.246700,0.000000,0.332592,0.000000,0.360645,0.342995,0.169190,0.339785,0.000000,0.666667,0.428571,0.428571,-8.055834,-34.951362,1
2,4180,0.184200,0.000000,0.168521,0.145990,0.318269,0.288125,0.175386,0.277219,0.000000,0.666667,0.428571,0.285714,-8.055424,-34.951597,1
3,8922,0.707155,0.446022,0.718534,0.490706,0.361142,0.451422,0.326812,0.331300,0.299672,0.166667,0.571429,0.428571,-8.049530,-34.953290,0
4,1117,0.121940,0.157226,0.161643,0.088497,0.252093,0.256320,0.087576,0.252854,0.000000,0.666667,0.428571,0.285714,-8.055380,-34.951736,1


In [26]:
X_train = df_train.drop(['ponto_id', 'indoor', 'lat', 'lon'], axis=1).values
y_train = df_train[['lat', 'lon']].values

X_test = df_test.drop(['ponto_id', 'indoor', 'lat', 'lon'], axis=1).values
y_test = df_test[['lat', 'lon']].values

In [27]:
model = RandomForestRegressor(
    n_estimators=10,
    max_depth=10,
    criterion='squared_error',
    min_samples_leaf=1,
    max_features=df_train.shape[1]
)

In [28]:
model.fit(X=X_train, y=y_train)

RandomForestRegressor(max_depth=10, max_features=16, n_estimators=10)

## Avaliando o modelo

#### Conjunto de Teste

In [29]:
y_pred_test_class = model.predict(X_test)

rmse = np.sqrt(root_mean_squared_error(y_test, y_pred_test_class))
print(f'RMSE: {rmse}')

RMSE: 0.01061455987429097


In [30]:
map = plot_folium_map(y_test, y_pred_test_class)
map

#### Avaliando o modelos para pontos específicos 

In [33]:
# Prevendo o valor de uma instância específica
idx = 75

X_unit = X_test[idx].reshape(1, -1)
y_unit = np.array([y_test[idx]])
y_pred_unit = model.predict(X_unit)

print(f"Pred Point: {y_pred_unit[0]}")
print(f"Real Point: {y_unit[0]}")

Pred Point: [ -8.055786 -34.951505]
Real Point: [ -8.055786 -34.951505]


In [34]:
map = plot_folium_map(y_unit, y_pred_unit)
map